<a href="https://colab.research.google.com/github/ele9996/keyword-spotting/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import os
from zipfile import ZipFile

In [43]:
#parser = argparse.ArgumentParser()
#parser.add_argument('--model', type=str, required=True, help='model name')
#parser.add_argument('--mfcc', action='store_true', help='use MFCCs')
#parser.add_argument('--silence', action='store_true', help='add silence')
#args = parser.parse_args()

mfcc=True
silence=True

In [44]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

#Data extraction from file

In [45]:
zip_path = tf.keras.utils.get_file(
        origin="http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip",
        fname='mini_speech_commands.zip',
        extract=True,
        cache_dir='.', cache_subdir='data')



#if args.silence is True:
if silence is True:
  zip_path_silence = tf.keras.utils.get_file(
        origin="https://github.com/ele9996/keyword-spotting/raw/main/silence.zip",
        fname='silence.zip',
        extract=True,
        cache_dir='./data', cache_subdir='mini_speech_commands')


data_dir = os.path.join('.', 'data', 'mini_speech_commands')
filenames = tf.io.gfile.glob(str(data_dir) + '/*/*')
filenames = tf.random.shuffle(filenames)
num_samples = len(filenames)

#if args.silence is True:
if silence is True:
    total = 9000
else:
    total = 8000

#Training, Validation and Test dataset definition

In [46]:
train_files = filenames[:int(total*0.8)]
val_files = filenames[int(total*0.8): int(total*0.9)]
test_files = filenames[int(total*0.9):]

In [47]:
LABELS = np.array(tf.io.gfile.listdir(str(data_dir)))
LABELS = LABELS[LABELS != 'README.md']

#Data preparation and preprocessing steps

In [48]:
class SignalGenerator:
    def __init__(self, labels, sampling_rate, frame_length, frame_step,
            num_mel_bins=None, lower_frequency=None, upper_frequency=None,
            num_coefficients=None, mfcc=False):
        self.labels = labels
        self.sampling_rate = sampling_rate
        self.frame_length = frame_length
        self.frame_step = frame_step
        self.num_mel_bins = num_mel_bins
        self.lower_frequency = lower_frequency
        self.upper_frequency = upper_frequency
        self.num_coefficients = num_coefficients
        num_spectrogram_bins = (frame_length) // 2 + 1

        if mfcc is True:
            self.linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
                    self.num_mel_bins, num_spectrogram_bins, self.sampling_rate,
                    self.lower_frequency, self.upper_frequency)
            self.preprocess = self.preprocess_with_mfcc
        else:
            self.preprocess = self.preprocess_with_stft

    def read(self, file_path):
        parts = tf.strings.split(file_path, os.path.sep)
        label = parts[-2]
        label_id = tf.argmax(label == self.labels)
        audio_binary = tf.io.read_file(file_path)
        audio, _ = tf.audio.decode_wav(audio_binary)
        audio = tf.squeeze(audio, axis=1)

        return audio, label_id

    def pad(self, audio):
        zero_padding = tf.zeros([self.sampling_rate] - tf.shape(audio), dtype=tf.float32)
        audio = tf.concat([audio, zero_padding], 0)
        audio.set_shape([self.sampling_rate])

        return audio

    def get_spectrogram(self, audio):
        stft = tf.signal.stft(audio, frame_length=self.frame_length,
                frame_step=self.frame_step, fft_length=self.frame_length)
        spectrogram = tf.abs(stft)

        return spectrogram

    def get_mfccs(self, spectrogram):
        mel_spectrogram = tf.tensordot(spectrogram,
                self.linear_to_mel_weight_matrix, 1)
        log_mel_spectrogram = tf.math.log(mel_spectrogram + 1.e-6)
        mfccs = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrogram)
        mfccs = mfccs[..., :self.num_coefficients]

        return mfccs

    def preprocess_with_stft(self, file_path):
        audio, label = self.read(file_path)
        audio = self.pad(audio)
        spectrogram = self.get_spectrogram(audio)
        spectrogram = tf.expand_dims(spectrogram, -1)
        spectrogram = tf.image.resize(spectrogram, [32, 32])

        return spectrogram, label

    def preprocess_with_mfcc(self, file_path):
        audio, label = self.read(file_path)
        audio = self.pad(audio)
        spectrogram = self.get_spectrogram(audio)
        mfccs = self.get_mfccs(spectrogram)
        mfccs = tf.expand_dims(mfccs, -1)

        return mfccs, label

    def make_dataset(self, files, train):
        ds = tf.data.Dataset.from_tensor_slices(files)
        ds = ds.map(self.preprocess, num_parallel_calls=4)
        ds = ds.batch(32)
        ds = ds.cache()
        if train is True:
            ds = ds.shuffle(100, reshuffle_each_iteration=True)

        return ds

In [49]:
STFT_OPTIONS = {'frame_length': 256, 'frame_step': 128, 'mfcc': False}
MFCC_OPTIONS = {'frame_length': 640, 'frame_step': 320, 'mfcc': True,
        'lower_frequency': 20, 'upper_frequency': 4000, 'num_mel_bins': 40,
        'num_coefficients': 10}

In [50]:
#if args.mfcc is True:
if mfcc is True:
    options = MFCC_OPTIONS
    strides = [2, 1]
else:
    options = STFT_OPTIONS
    strides = [2, 2]


In [51]:
generator = SignalGenerator(LABELS, 16000, **options)
train_ds = generator.make_dataset(train_files, True)
val_ds = generator.make_dataset(val_files, False)
test_ds = generator.make_dataset(test_files, False)

#if args.silence is True:
if silence is True:
    units = 9
else:
    units = 8

#Model application

###MPL

In [52]:
if (mfcc==True):
  path= './callback_val_chkp/mfcc_mpl'
else:
  path= './callback_val_chkp/stft_mpl'
  
cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=path,
    # './callback_test_chkp/chkp_best',
    monitor='val_loss',
    verbose=0, 
    save_best_only=True,
    # save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch'
)



In [53]:
#MLP Model definition
mlp_model = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dense(units=10) #if silence 10
])

In [54]:
#Training, evaluation and testing with mlp model


mlp_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,optimizer='adam', metrics=['accuracy'])
mlp_model.fit(train_ds, epochs=20, validation_data=val_ds,callbacks=[cp_callback])



Epoch 1/20
225/225 [==============================] - 116s 515ms/step - loss: 1.5658 - accuracy: 0.4961 - val_loss: 1.0272 - val_accuracy: 0.6567
Epoch 2/20
225/225 [==============================] - 1s 7ms/step - loss: 0.8455 - accuracy: 0.7013 - val_loss: 0.7838 - val_accuracy: 0.7222
Epoch 3/20
225/225 [==============================] - 1s 4ms/step - loss: 0.6284 - accuracy: 0.7799 - val_loss: 0.7847 - val_accuracy: 0.7256
Epoch 4/20
225/225 [==============================] - 1s 7ms/step - loss: 0.5289 - accuracy: 0.8136 - val_loss: 0.7719 - val_accuracy: 0.7433
Epoch 5/20
225/225 [==============================] - 2s 8ms/step - loss: 0.4239 - accuracy: 0.8486 - val_loss: 0.7100 - val_accuracy: 0.7600
Epoch 6/20
225/225 [==============================] - 1s 4ms/step - loss: 0.3534 - accuracy: 0.8769 - val_loss: 0.7503 - val_accuracy: 0.7678
Epoch 7/20
225/225 [==============================] - 1s 6ms/step - loss: 0.3346 - accuracy: 0.8835 - val_loss: 0.7055 - val_accuracy: 0.7622
Ep

In [55]:
#model evaluation on validation set
mlp_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam')
val_loss= mlp_model.evaluate(val_ds)

29/29 [==============================] - 0s 1ms/step - loss: 1.5380


In [56]:
test_loss= mlp_model.evaluate(test_ds)

29/29 [==============================] - 20s 695ms/step - loss: 1.6096


###CNN

In [57]:
if (mfcc==True):
  path= './callback_val_chkp/mfcc_cnn'
else:
  path= './callback_val_chkp/stft_cnn'

cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=path,
    # './callback_test_chkp/chkp_best',
    monitor='val_loss',
    verbose=0, 
    save_best_only=True,
    # save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch'
)


In [58]:
#CNN_2D Model definition

cnn_model = keras.Sequential([
    keras.layers.Conv2D(filters=128,kernel_size=[3,3],strides=strides, use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.Conv2D(filters=128,kernel_size=[3,3],strides=[1, 1], use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.Conv2D(filters=128,kernel_size=[3,3],strides=[1,1], use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(units=10) #10 if silence
])

In [59]:
#Training, evaluation and testing with mlp model
cnn_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,optimizer='adam', metrics=['accuracy'])
cnn_model.fit(train_ds, epochs=20, validation_data=val_ds,callbacks=[cp_callback])

Epoch 1/20
225/225 [==============================] - 34s 153ms/step - loss: 1.1533 - accuracy: 0.6503 - val_loss: 0.7114 - val_accuracy: 0.7967
Epoch 2/20
225/225 [==============================] - 35s 154ms/step - loss: 0.5800 - accuracy: 0.8517 - val_loss: 0.4783 - val_accuracy: 0.8644
Epoch 3/20
225/225 [==============================] - 35s 154ms/step - loss: 0.4034 - accuracy: 0.8994 - val_loss: 0.3715 - val_accuracy: 0.8978
Epoch 4/20
225/225 [==============================] - 34s 153ms/step - loss: 0.3161 - accuracy: 0.9175 - val_loss: 0.2821 - val_accuracy: 0.9167
Epoch 5/20
225/225 [==============================] - 33s 147ms/step - loss: 0.2717 - accuracy: 0.9265 - val_loss: 0.2992 - val_accuracy: 0.9089
Epoch 6/20
225/225 [==============================] - 34s 153ms/step - loss: 0.2252 - accuracy: 0.9394 - val_loss: 0.2449 - val_accuracy: 0.9244
Epoch 7/20
225/225 [==============================] - 33s 145ms/step - loss: 0.1957 - accuracy: 0.9447 - val_loss: 0.2610 - val_ac

In [60]:
#model evaluation on validation set
cnn_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam')
val_loss= cnn_model.evaluate(val_ds)

29/29 [==============================] - 1s 27ms/step - loss: 0.9264


In [61]:
#model test on the test set
test_loss= cnn_model.evaluate(test_ds)

29/29 [==============================] - 1s 26ms/step - loss: 0.8861


###DS-CNN

In [62]:
if (mfcc==True):
  path= './callback_val_chkp/mfcc_DS-CNN'
else:
  path= './callback_val_chkp/stft_DS-CNN'

cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=path,
    # './callback_test_chkp/chkp_best',
    monitor='val_loss',
    verbose=0, 
    save_best_only=True,
    # save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch'
)


In [63]:
ds_cnn_model = keras.Sequential([
    keras.layers.Conv2D(filters=256,kernel_size=[3,3],strides=strides, use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.DepthwiseConv2D(kernel_size=[3,3],strides=[1, 1], use_bias=False),
    keras.layers.Conv2D(filters=256,kernel_size=[1,1],strides=[1,1], use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.DepthwiseConv2D(kernel_size=[3,3],strides=[1, 1], use_bias=False),
    keras.layers.Conv2D(filters=256,kernel_size=[1,1],strides=[1,1], use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(units=10) #10 if silence
])

In [64]:
#Training, evaluation and testing with mlp model
ds_cnn_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,optimizer='adam', metrics=['accuracy'])
ds_cnn_model.fit(train_ds, epochs=20, validation_data=val_ds,callbacks=[cp_callback])

Epoch 1/20
225/225 [==============================] - 29s 130ms/step - loss: 1.1212 - accuracy: 0.6535 - val_loss: 0.6597 - val_accuracy: 0.8189
Epoch 2/20
225/225 [==============================] - 29s 131ms/step - loss: 0.5511 - accuracy: 0.8464 - val_loss: 0.4320 - val_accuracy: 0.8856
Epoch 3/20
225/225 [==============================] - 29s 130ms/step - loss: 0.3874 - accuracy: 0.8938 - val_loss: 0.3091 - val_accuracy: 0.9233
Epoch 4/20
225/225 [==============================] - 29s 129ms/step - loss: 0.3099 - accuracy: 0.9131 - val_loss: 0.2610 - val_accuracy: 0.9289
Epoch 5/20
225/225 [==============================] - 29s 131ms/step - loss: 0.2519 - accuracy: 0.9285 - val_loss: 0.2592 - val_accuracy: 0.9278
Epoch 6/20
225/225 [==============================] - 28s 122ms/step - loss: 0.2226 - accuracy: 0.9336 - val_loss: 0.2602 - val_accuracy: 0.9178
Epoch 7/20
225/225 [==============================] - 29s 131ms/step - loss: 0.1933 - accuracy: 0.9439 - val_loss: 0.2306 - val_ac

In [65]:
#model evaluation on validation set
ds_cnn_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam')
val_loss= ds_cnn_model.evaluate(val_ds)

29/29 [==============================] - 1s 26ms/step - loss: 0.8695


In [66]:
#model test on the test set
test_loss= ds_cnn_model.evaluate(test_ds)

29/29 [==============================] - 1s 26ms/step - loss: 0.8204


In [67]:
!zip -r silence_model.zip {"./callback_val_chkp"}

  adding: callback_val_chkp/ (stored 0%)
  adding: callback_val_chkp/stft_mpl/ (stored 0%)
  adding: callback_val_chkp/stft_mpl/assets/ (stored 0%)
  adding: callback_val_chkp/stft_mpl/saved_model.pb (deflated 89%)
  adding: callback_val_chkp/stft_mpl/variables/ (stored 0%)
  adding: callback_val_chkp/stft_mpl/variables/variables.index (deflated 66%)
  adding: callback_val_chkp/stft_mpl/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: callback_val_chkp/stft_cnn/ (stored 0%)
  adding: callback_val_chkp/stft_cnn/assets/ (stored 0%)
  adding: callback_val_chkp/stft_cnn/saved_model.pb (deflated 91%)
  adding: callback_val_chkp/stft_cnn/variables/ (stored 0%)
  adding: callback_val_chkp/stft_cnn/variables/variables.index (deflated 69%)
  adding: callback_val_chkp/stft_cnn/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: callback_val_chkp/mfcc_mpl/ (stored 0%)
  adding: callback_val_chkp/mfcc_mpl/assets/ (stored 0%)
  adding: callback_val_chkp/mfcc_mpl/saved_mod

In [68]:
!ls

callback_val_chkp  data  sample_data  silence_model.zip


In [69]:
try:
  from google.colab import files
  files.download('./silence_model.zip')
except ImportError:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>